## YOLO - Intruder Detection

### Required Dependencies

In [1]:
# Supress warnings for entire notebook
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Import the video processing and deep learning modules
import cv2
import numpy as np
import os
import threading
from ultralytics import YOLO
from datetime import datetime
import time
import json

In [3]:
# # Get firebase backend storage functions
import firebase_admin
from firebase_admin import credentials, storage, firestore

### YOLO - Setup

In [4]:
# Load YOLO model
yolo_model = YOLO("yolov8n.pt")  # Ensure you have the model downloaded

In [5]:
# Create a folder for saving intruder images
INTRUDER_FOLDER = "intruders"
os.makedirs(INTRUDER_FOLDER, exist_ok=True)

In [6]:
# Intruder detection state
state = 'stop'   # 'stop', 'start'
last_intruder_time = 0    # intruder time tracker
follow_update = False     # update flag to tick intruder time

# Flag to prevent multiple burst threads running at the same time
burst_in_progress = False

### YOLO - Functions

In [7]:
# Function to detect people using YOLO
def detect_persons(frame):
    results = yolo_model(frame)  # Run YOLO
    persons = []

    for result in results:
        for box, class_id, conf in zip(result.boxes.xyxy, result.boxes.cls, result.boxes.conf):  # Extract confidence score
            if int(class_id) == 0:  # Class 0 in COCO is "person"
                x1, y1, x2, y2 = map(int, box[:4])
                persons.append({
                    "bbox": (x1, y1, x2, y2),  # Bounding box
                    "confidence": conf.item()   # Confidence score
                })

    return persons

In [8]:
# Helper function to save a frame
def save_intruder_frame(frame, prefix=""):
    # Save files to local disk
    filename = f"{INTRUDER_FOLDER}/Intruder_{prefix}_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.jpg"
    cv2.imwrite(filename, frame)
    print(f"Saved: {filename}")
    
    # Optional: Call FIREBASE cloud upload function here
    handle_intruder_detection(filename, confidence)

In [9]:
# First Burst Function - 5 photos immediately, 300ms apart
def first_burst(frame):
    global state, burst_in_progress
    
    if state != 'stop' or burst_in_progress:
        return  # Already capturing or burst in progress
    
    state = 'start'
    burst_in_progress = True  # Set the flag to indicate burst is in progress
    print("Intruder detected - Starting first burst capture")

    # Background thread for burst capture
    def capture_burst():
        global burst_in_progress
        
        for i in range(5):
            save_intruder_frame(frame, f"{i+1}_burst")
            time.sleep(0.3)  # This sleep only affects the burst thread, not the main loop

        burst_in_progress = False  # Reset flag after burst capture is complete

    # Start a thread for the burst capture
    threading.Thread(target=capture_burst, daemon=True).start()

In [10]:
# Follow-up Function - 1 photo every 2 seconds
def follow_up(frame):
    global state, last_intruder_time, follow_update
    
    if state != 'start':
        return  # No active capture
    
    if time.time() - last_intruder_time >= 2:  # 2 seconds interval
        save_intruder_frame(frame, "followup")
        last_intruder_time = time.time()

In [11]:
# Exit Function - stop after 5 seconds of no intruder detected
def exit_capture():
    print("... INSIDE EXIT...")
    global state, follow_update
    
    print("No intruder for 5 seconds - Stopping capture")
    state = 'stop'
    follow_update = False

### Firebase - Functions

In [12]:
# # Upload image to Firebase Storage
def upload_image(image_path):
    bucket = storage.bucket()
    blob = bucket.blob(f"intruders/{os.path.basename(image_path)}")
    blob.upload_from_filename(image_path)
    blob.make_public()  # Makes the image URL publicly accessible (optional)

    return blob.public_url

In [13]:
# # Save Metadata to Firestore
def save_metadata(image_url, confidence):
    doc_ref = db.collection('intruder_logs').add({
        'imageUrl': image_url,
        'timestamp': datetime.now().isoformat(),
        'confidence': confidence
    })

In [14]:
# # Full Function to Call from Detection Code
def handle_intruder_detection(image_path, confidence):
    image_url = upload_image(image_path)
    save_metadata(image_url, confidence)
    print(f"Intruder saved: {image_url}")

### Firebase - Setup

In [15]:
# # Initialize Firebase App
cred = credentials.Certificate("firebase_key.json")
firebase_admin.initialize_app(cred, {
    'storageBucket': 'intruder-detection-syste-3e8f1.firebasestorage.app'
})
db = firestore.client()

### Intruder Detection Loop - Frame by Frame Analysis

In [16]:
# Initialize webcam
cap = cv2.VideoCapture(0)

In [17]:
# Start analyzing the frames until quit
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    persons = detect_persons(frame)
    
    if persons:  # Intruder Detected
#         print("PERSON DETECTED ...")
        if follow_update == False:
            last_intruder_time = time.time()
            follow_update = True

        if state == 'stop':
#             print("FIRST BURST ...")
            first_burst(frame)  # Trigger burst when intruder detected for first time
        
        # Draw boxes and labels
        for person in persons:
            x1, y1, x2, y2 = person["bbox"]
            confidence = person["confidence"]
 
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
            text = f"Intruder Detected: {confidence:.2f}"
            cv2.putText(frame, text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
        
        # Continuous follow-up capture if burst is done
        if state == 'start' and burst_in_progress == False:
            follow_up(frame)

    else:  # No intruder detected
        if state == 'start' and time.time() - last_intruder_time > 5:
            exit_capture()
    
    # Display the live feed with annotations
    cv2.imshow("Intruder Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break
    
cap.release()
cv2.destroyAllWindows()


0: 480x640 2 persons, 1 knife, 245.8ms
Speed: 9.2ms preprocess, 245.8ms inference, 3.5ms postprocess per image at shape (1, 3, 480, 640)
Intruder detected - Starting first burst capture
Saved: intruders/Intruder_1_burst_2025-03-22_21-32-51.jpg

0: 480x640 2 persons, 112.6ms
Speed: 7.2ms preprocess, 112.6ms inference, 1.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 120.2ms
Speed: 3.9ms preprocess, 120.2ms inference, 1.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 109.4ms
Speed: 3.2ms preprocess, 109.4ms inference, 1.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 103.4ms
Speed: 3.5ms preprocess, 103.4ms inference, 2.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 99.7ms
Speed: 2.5ms preprocess, 99.7ms inference, 1.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 104.3ms
Speed: 2.4ms preprocess, 104.3ms inference, 1.6ms postprocess per image at shape 


0: 480x640 2 persons, 94.0ms
Speed: 2.4ms preprocess, 94.0ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 93.1ms
Speed: 2.0ms preprocess, 93.1ms inference, 1.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 82.6ms
Speed: 2.0ms preprocess, 82.6ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 86.2ms
Speed: 1.4ms preprocess, 86.2ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 85.0ms
Speed: 1.6ms preprocess, 85.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 72.7ms
Speed: 1.8ms preprocess, 72.7ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 75.3ms
Speed: 2.1ms preprocess, 75.3ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 77.8ms
Speed: 2.1ms preprocess, 77.8ms inference, 1.9ms postprocess per image at shape (


0: 480x640 2 persons, 124.3ms
Speed: 2.2ms preprocess, 124.3ms inference, 2.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 122.5ms
Speed: 2.1ms preprocess, 122.5ms inference, 2.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 125.8ms
Speed: 2.6ms preprocess, 125.8ms inference, 2.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 127.9ms
Speed: 3.2ms preprocess, 127.9ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 118.5ms
Speed: 2.6ms preprocess, 118.5ms inference, 2.2ms postprocess per image at shape (1, 3, 480, 640)
Saved: intruders/Intruder_followup_2025-03-22_21-33-10.jpg
Intruder saved: https://storage.googleapis.com/intruder-detection-syste-3e8f1.firebasestorage.app/intruders/Intruder_followup_2025-03-22_21-33-10.jpg

0: 480x640 2 persons, 214.5ms
Speed: 4.5ms preprocess, 214.5ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 per


0: 480x640 2 persons, 129.6ms
Speed: 2.5ms preprocess, 129.6ms inference, 2.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 134.0ms
Speed: 3.3ms preprocess, 134.0ms inference, 2.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 persons, 141.1ms
Speed: 3.1ms preprocess, 141.1ms inference, 2.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 persons, 1 dog, 136.2ms
Speed: 3.1ms preprocess, 136.2ms inference, 2.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 persons, 128.3ms
Speed: 3.0ms preprocess, 128.3ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 123.8ms
Speed: 2.4ms preprocess, 123.8ms inference, 2.3ms postprocess per image at shape (1, 3, 480, 640)
Saved: intruders/Intruder_followup_2025-03-22_21-33-20.jpg
Intruder saved: https://storage.googleapis.com/intruder-detection-syste-3e8f1.firebasestorage.app/intruders/Intruder_followup_2025-03-22_21-33-20.jpg

0: 480x64


0: 480x640 2 persons, 115.6ms
Speed: 2.5ms preprocess, 115.6ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 110.8ms
Speed: 2.2ms preprocess, 110.8ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 112.9ms
Speed: 2.5ms preprocess, 112.9ms inference, 2.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 126.0ms
Speed: 3.4ms preprocess, 126.0ms inference, 2.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 111.5ms
Speed: 2.8ms preprocess, 111.5ms inference, 2.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 119.1ms
Speed: 4.0ms preprocess, 119.1ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 109.1ms
Speed: 2.7ms preprocess, 109.1ms inference, 1.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 persons, 1 cat, 115.1ms
Speed: 2.8ms preprocess, 115.1ms inference, 2.4ms postproce


0: 480x640 2 persons, 113.8ms
Speed: 2.1ms preprocess, 113.8ms inference, 2.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 122.4ms
Speed: 2.1ms preprocess, 122.4ms inference, 2.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 114.9ms
Speed: 3.6ms preprocess, 114.9ms inference, 1.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 110.0ms
Speed: 1.6ms preprocess, 110.0ms inference, 2.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 118.4ms
Speed: 2.0ms preprocess, 118.4ms inference, 2.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 136.0ms
Speed: 2.7ms preprocess, 136.0ms inference, 3.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 138.7ms
Speed: 4.0ms preprocess, 138.7ms inference, 1.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 109.0ms
Speed: 2.4ms preprocess, 109.0ms inference, 1.5ms postprocess per 


0: 480x640 2 persons, 124.6ms
Speed: 2.9ms preprocess, 124.6ms inference, 1.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 124.7ms
Speed: 3.1ms preprocess, 124.7ms inference, 2.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 129.6ms
Speed: 4.0ms preprocess, 129.6ms inference, 1.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 131.1ms
Speed: 2.2ms preprocess, 131.1ms inference, 2.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 persons, 113.7ms
Speed: 3.0ms preprocess, 113.7ms inference, 2.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 121.5ms
Speed: 2.8ms preprocess, 121.5ms inference, 2.8ms postprocess per image at shape (1, 3, 480, 640)
Saved: intruders/Intruder_followup_2025-03-22_21-33-51.jpg
Intruder saved: https://storage.googleapis.com/intruder-detection-syste-3e8f1.firebasestorage.app/intruders/Intruder_followup_2025-03-22_21-33-51.jpg

0: 480x640 2 per